In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,roc_auc_score,ConfusionMatrixDisplay,precision_score,recall_score
import numpy as np
from scipy import signal
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
main_dir = 'drive/MyDrive/23Spring-MLMA/isolated/'

patient = ["001","002","003","004","005","006"]
test_split = [["01","02"],["03","04"],["05","06"],["07","08"],["09","10"]]
val_split = ['03','01',"04","05","06"]

os.listdir(main_dir)

['002',
 '005',
 '004',
 '003',
 '001',
 '006',
 'User_dependent',
 'Isolated Model',
 'User_independent',
 'XGboost0.pkl',
 'XGboost1.pkl',
 'XGboost2.pkl',
 'XGboost3.pkl',
 'XGboost4.pkl',
 'XGboost5.pkl',
 'XGboost_User_Independent.pkl']

In [4]:
# -*- coding: utf-8 -*-
"""preprocess.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1_HGxPyUOCJ3wnMfPbXl7sqWSz-jsCMGe
"""
# input: data (type: numpy array)(shape: (time, 2))
# output: data (type: numpy array)(shape: (100, 2))
import numpy as np
from scipy import signal

def preprocess(data):
  # Delete silent part
  data = data.astype(float)
  n_size = 50
  n_len = int(data.shape[0]/n_size)
  std_data = np.zeros((n_size, 2))
  for i in range(n_size):
    seg_data_x = data[i*n_len:i*n_len+n_len, 0]
    seg_data_y = data[i*n_len:i*n_len+n_len, 1]
    std_data[i, 0] = np.std(seg_data_x)
    std_data[i, 1] = np.std(seg_data_y)
  pass_threshold = 1
  pass_idx_x = np.where(std_data[:,0] >= pass_threshold)[0]
  pass_idx_y = np.where(std_data[:,1] >= pass_threshold)[0]
  if len(pass_idx_x) == 0:
    start_idx = max(0, pass_idx_y[0] - 1)
    end_idx = min(data.shape[0],pass_idx_y[-1] + 1)
  elif len(pass_idx_y) == 0:
    start_idx = max(0, pass_idx_x[0] - 1)
    end_idx = min(data.shape[0],pass_idx_x[-1] + 1)
  else:
    start_idx = max(0,min(pass_idx_x[0], pass_idx_y[0]) - 1)
    end_idx = min(data.shape[0],max(pass_idx_x[-1], pass_idx_y[-1]) + 1)
  
  # resample to 100 data points
  data = signal.resample(data[start_idx*n_len:end_idx*n_len, :], 100, axis=0)
  # scale
  data = (data - data.min(axis=0, keepdims=True))/(data.max(axis=0, keepdims=True) - data.min(axis=0, keepdims=True))
  return data

# get file list for 1 patient
def get_file_list(path):
    file_list = []
    label = []
    for i in os.listdir(path):
        file_list.append(i)
        l = int(i.split("_")[-1].split(".")[0])-1
        label.append(l)
    return label,file_list

# get feature from 1 file and preprocess
def get_feature(path):
    f = []
    a = pd.read_csv(path,names=["vertical","horizontal"])
    a = np.array(a)
    a = preprocess(a)
    for j in a[:,0]:
        f.append(j)        
    for j in a[:,1]:
        f.append(j)
    return f

def evaluate(y_true, y_pred,y_prob):
    f1_micro = f1_score(y_true, y_pred,average = 'micro')
    f1_macro = f1_score(y_true, y_pred,average = 'macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true,y_prob,multi_class="ovr",average="micro")

    return f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc

# Split on patient level - User-independent


> 6-fold cross-validation





In [5]:
# self identified test_split
def my_train_test_split_user_independent(test_patient,val_patient,train_patient):
    X_test = []
    X_train = []
    X_val = []
    y_val = []
    y_train = []
    y_test = []
    for p in train_patient:
        path = str(main_dir+p+"/isolated_strokes/")#str("../../../../data/isolated 2/"+p+"/isolated_strokes/")
        label,file_list = get_file_list(path)
        for i in range(len(file_list)):
            file = file_list[i]
            file_label = label[i]
            feature = get_feature(str(path+file))
            X_train.append(feature)
            y_train.append(file_label)
        
    path = str(main_dir+test_patient+"/isolated_strokes/")#str("../../../../data/isolated 2/"+test_patient+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
        file = file_list[i]
        file_label = label[i]
        feature = get_feature(str(path+file))
        X_test.append(feature)
        y_test.append(file_label)
    
    path = str(main_dir+val_patient+"/isolated_strokes/")# str("../../../../data/isolated 2/"+val_patient+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
        file = file_list[i]
        file_label = label[i]
        feature = get_feature(str(path+file))
        X_val.append(feature)
        y_val.append(file_label)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    return X_train,X_test, X_val,y_val,y_train,y_test

In [6]:
from sklearn.ensemble import RandomForestClassifier
def train(train_feature,train_label,test_feature,test_label,n,d):
    # 定义模型
    model = RandomForestClassifier(n_estimators=n,max_depth=d, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)#,eval_set=[(test_feature, test_label)])

    # 预测测试集
    y_pred_p = model.predict_proba(test_feature)
    y_pred = model.predict(test_feature)

    # 计算metrics
    f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc = evaluate(test_label, y_pred,y_pred_p)

    return f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc,model

In [8]:
test_patien_list = ["001","002","003","004","005","006"]
val_patient_list = ["002","003","004","005","006","001"]
output_pck = []
F1 = 0
D = [2,4,6,8,10,50]
N = [10,100,500,1000]
output = dict()

for t in range(len(test_patien_list)):
    train_patient= ["001","002","003","004","005","006"]
    test_patient = test_patien_list[t]
    val_patient = val_patient_list[t]
    train_patient.remove(test_patient)
    train_patient.remove(val_patient)
    print(test_patient,val_patient,train_patient)
    X_train,X_test, X_val,y_val,y_train,y_test = my_train_test_split_user_independent(test_patient,val_patient,train_patient)

    # output = dict()
    # output["test_split"] = t
    # grid_search_output = grid_search(X_train,X_test, X_val,y_val,y_train,y_test)
    # output["grid_search"] = grid_search_output
    # output_pck.append(output)
    for n in N:
      for d in D:
        output = dict()
        f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc,model = train(X_train,y_train,X_val,y_val,n,d)
        output['n']=n;output['d']=d;output["test_split"] =t;output['model']=model
        output['f1_micro']=f1_micro;output['f1_macro']=f1_macro;output['precision_micro']=precision_micro;output['precision_macro']=precision_macro
        output['recall_micro']=recall_micro;output['recall_macro']=recall_macro;output['acc']=acc;output['auc']=auc
        # print(output)
        output_pck.append(output)
    

# 保存模型
pickle.dump(output_pck, open(main_dir+"RF_User_Independent"+".pkl", 'wb'))

001 002 ['003', '004', '005', '006']
002 003 ['001', '004', '005', '006']
003 004 ['001', '002', '005', '006']
004 005 ['001', '002', '003', '006']
005 006 ['001', '002', '003', '004']
006 001 ['002', '003', '004', '005']


In [9]:
f1=np.zeros((len(N),len(D)))
accu=np.zeros((len(N),len(D)))
for n in range(len(N)):
  for d in range(len(D)):
    for i in range(len(output_pck)):
      if output_pck[i]['n']==N[n] and output_pck[i]['d']==D[d]:
        f1[n][d]+=output_pck[i]['f1_macro']
        accu[n][d]+=output_pck[i]['acc']
f1/=len(test_patien_list)
accu/=len(test_patien_list)

In [10]:
print(f1)
f1.argmax()

[[0.32212025 0.55713828 0.59652928 0.61556248 0.60170664 0.59399642]
 [0.44540797 0.59296086 0.63206358 0.63793995 0.63406611 0.64567177]
 [0.47797939 0.59661959 0.62549187 0.63087239 0.63376363 0.64960396]
 [0.47987948 0.60071377 0.63127595 0.62938879 0.62827233 0.64480612]]


17

In [11]:
print(accu)
accu.argmax()

[[0.39390044 0.58297731 0.61488845 0.62857148 0.61340792 0.60509717]
 [0.50971037 0.61617385 0.64388275 0.64939296 0.64940425 0.66050426]
 [0.53324151 0.61481921 0.6383161  0.64384888 0.65085034 0.66460243]
 [0.53466464 0.61756274 0.64247129 0.64249405 0.64531756 0.66053813]]


17

max_depth=50, 
n_estimators=500

In [12]:
f1t=0;acct=0;pret=0;rect=0;auct=0;c=0
f1tm=0;pretm=0;rectm=0

for t in range(len(test_patien_list)):
    train_patient= ["001","002","003","004","005","006"]
    test_patient = test_patien_list[t]
    val_patient = val_patient_list[t]
    train_patient.remove(test_patient)
    train_patient.remove(val_patient)
    print(test_patient,val_patient,train_patient)
    X_train,X_test, X_val,y_val,y_train,y_test = my_train_test_split_user_independent(test_patient,val_patient,train_patient)

    for i in range(len(output_pck)):
      if output_pck[i]['test_split']==t and output_pck[i]['n']==500 and output_pck[i]['d']==50:
        # 定义模型
        model = output_pck[i]['model']
        # 预测测试集
        y_pred_p = model.predict_proba(X_test)
        y_pred = model.predict(X_test)
        # 计算metrics
        f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc = evaluate(y_test, y_pred,y_pred_p)
        f1t+=f1_macro;pret+=precision_macro;acct+=acc;rect+=recall_macro;auct+=auc;c+=1
        f1tm+=f1_micro;pretm+=precision_micro;rectm+=recall_micro

f1t/=c;acct/=c;pret/=c;rect/=c
f1tm/=c;pretm/=c;rectm/=c;auct/=c

001 002 ['003', '004', '005', '006']
002 003 ['001', '004', '005', '006']
003 004 ['001', '002', '005', '006']
004 005 ['001', '002', '003', '006']
005 006 ['001', '002', '003', '004']
006 001 ['002', '003', '004', '005']


In [13]:
print("test accuracy:",acct)
print("test f1 macro:",f1t)
print("test f1 micro:",f1tm)
print("test precision macro:",pret)
print("test precision micro:",pretm)
print("test recall macro:",rect)
print("test recall micro:",rectm)
print("test auc:",auct)

test accuracy: 0.6605608996070993
test f1 macro: 0.6365419511142089
test f1 micro: 0.6605608996070993
test precision macro: 0.6941115819069895
test precision micro: 0.6605608996070993
test recall macro: 0.6613355780022447
test recall micro: 0.6605608996070993
test auc: 0.9510755556515541


# User-dependent


> 5-fold cross-validation





In [14]:
# self identified test_split
def my_train_test_split_user_dependent(path,test_split,val_split,file_list,label):
    X_test = []
    X_train = []
    X_val = []
    y_val = []
    y_train = []
    y_test = []
    for f in range(len(file_list)):
        file = file_list[f]
        file_label = label[f]
        #print(file)
        feature = get_feature(str(path+file))
        #print(file.split('_')[2],file_label)
        if file.split('_')[2] in test_split:
            X_test.append(feature)
            y_test.append(file_label)
            #print(file,len(feature))
        elif file.split('_')[2] == val_split:
            X_val.append(feature)
            y_val.append(file_label)
            # print("val ",file)
        else:
            X_train.append(feature)
            y_train.append(file_label)
            # print("train ",file)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_val = np.array(X_val)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    y_test = np.array(y_test)
    
    return X_train,X_test,X_val,y_val, y_train,y_test

In [15]:
output_pck_pck=[]
for p in patient:
    print(p)
    output_pck = []
    path = str(main_dir+p+"/isolated_strokes/")#str("../../../../data/isolated 2/"+p+"/isolated_strokes/")
    for t in range(len(test_split)):
        print(test_split[t])
        label,file_list = get_file_list(path)
        X_train,X_test,X_val,y_val, y_train,y_test = my_train_test_split_user_dependent(path,test_split[t],val_split[t],file_list,label)
        print(X_val.shape,y_val.shape)
         # build model
        # set parameter 
        # output = dict()
        # output["test_split"] = t
        # grid_search_output = grid_search(X_train,X_test, X_val,y_val,y_train,y_test)
        # output["grid_search"] = grid_search_output
        # output_pck.append(output)

        for n in N:
          for d in D:
            output = dict()
            f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc,model = train(X_train,y_train,X_val,y_val,n,d)
            output['n']=n;output['d']=d;output["test_split"] =t;output["patient"] =p;output['model']=model
            output['f1_micro']=f1_micro;output['f1_macro']=f1_macro;output['precision_micro']=precision_micro;output['precision_macro']=precision_macro
            output['recall_micro']=recall_micro;output['recall_macro']=recall_macro;output['acc']=acc;output['auc']=auc
            # print(output)
            output_pck.append(output)
            output_pck_pck.append(output)


        
    # with open(str(main_dir+"/User_dependent/patient_"+p+"_XGboost.pck"), "wb") as output_file:
    #     pickle.dump(output_pck, output_file)
    pickle.dump(output_pck, open(str(main_dir+"User_dependent/patient_"+p+"_RF.pck"), "wb"))
        

001
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)
002
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)
003
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)
004
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)
005
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)
006
['01', '02']
(12, 200) (12,)
['03', '04']
(12, 200) (12,)
['05', '06']
(12, 200) (12,)
['07', '08']
(12, 200) (12,)
['09', '10']
(12, 200) (12,)


In [18]:
position=[]
for p in patient:
  print(p)
  f1=np.zeros((len(N),len(D)))
  accu=np.zeros((len(N),len(D)))
  for n in range(len(N)):
    for d in range(len(D)):
      for i in range(len(output_pck_pck)):
        if output_pck_pck[i]['n']==N[n] and output_pck_pck[i]['d']==D[d] and output_pck_pck[i]['patient']==p:
          f1[n][d]+=output_pck_pck[i]['f1_macro']
          accu[n][d]+=output_pck_pck[i]['acc']
  f1/=len(test_split)
  accu/=len(test_split)
  # print(f1)
  # print(f1.argmax())
  print(accu)
  print(accu.argmax())
  position.append(accu.argmax())

001
[[0.56666667 0.7        0.78333333 0.76666667 0.75       0.75      ]
 [0.71666667 0.76666667 0.76666667 0.76666667 0.76666667 0.76666667]
 [0.73333333 0.76666667 0.76666667 0.78333333 0.78333333 0.78333333]
 [0.71666667 0.73333333 0.76666667 0.76666667 0.76666667 0.76666667]]
2
002
[[0.65       0.85       0.88333333 0.88333333 0.9        0.9       ]
 [0.9        0.93333333 0.91666667 0.95       0.93333333 0.93333333]
 [0.86666667 0.93333333 0.91666667 0.91666667 0.91666667 0.91666667]
 [0.88333333 0.93333333 0.93333333 0.93333333 0.93333333 0.93333333]]
9
003
[[0.56666667 0.75       0.75       0.73333333 0.75       0.75      ]
 [0.73333333 0.75       0.81666667 0.83333333 0.8        0.8       ]
 [0.71666667 0.78333333 0.8        0.8        0.8        0.8       ]
 [0.75       0.78333333 0.8        0.81666667 0.81666667 0.81666667]]
9
004
[[0.45       0.65       0.65       0.68333333 0.65       0.65      ]
 [0.65       0.7        0.68333333 0.71666667 0.71666667 0.7       ]
 [0.66666

In [17]:
f1tl=[];acctl=[];pretl=[];rectl=[]
f1tml=[];pretml=[];rectml=[];auctl=[]
for pi in range(len(patient)):
    p=patient[pi]
    # print(p)
    path = str(main_dir+p+"/isolated_strokes/")#str("../../../../data/isolated 2/"+p+"/isolated_strokes/")
    f1t=0;acct=0;pret=0;rect=0;auct=0;c=0
    f1tm=0;pretm=0;rectm=0
    for t in range(len(test_split)):
        # print(test_split[t])
        label,file_list = get_file_list(path)
        X_train,X_test,X_val,y_val, y_train,y_test = my_train_test_split_user_dependent(path,test_split[t],val_split[t],file_list,label)
        # print(X_val.shape,y_val.shape)

        for i in range(len(output_pck_pck)):
          if position[pi]==5:
            if output_pck_pck[i]['test_split']==t and output_pck_pck[i]['patient']==p and output_pck_pck[i]['n']==100 and output_pck_pck[i]['d']==2:
              # 定义模型
              model = output_pck_pck[i]['model']
              # 预测测试集
              y_pred_p = model.predict_proba(X_test)
              y_pred = model.predict(X_test)
              # 计算metrics
              f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc = evaluate(y_test, y_pred,y_pred_p)
              f1t+=f1_macro;pret+=precision_macro;acct+=acc;rect+=recall_macro;auct+=auc;c+=1
              f1tm+=f1_micro;pretm+=precision_micro;rectm+=recall_micro
          elif position[pi]==6:
            if output_pck_pck[i]['test_split']==t and output_pck_pck[i]['patient']==p and output_pck_pck[i]['n']==100 and output_pck_pck[i]['d']==4:
              # 定义模型
              model = output_pck_pck[i]['model']
              # 预测测试集
              y_pred_p = model.predict_proba(X_test)
              y_pred = model.predict(X_test)
              # 计算metrics
              f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc,auc = evaluate(y_test, y_pred,y_pred_p)
              f1t+=f1_macro;pret+=precision_macro;acct+=acc;rect+=recall_macro;auct+=auc;c+=1
              f1tm+=f1_micro;pretm+=precision_micro;rectm+=recall_micro

    f1t/=c;acct/=c;pret/=c;rect/=c
    f1tm/=c;pretm/=c;rectm/=c;auct/=c
    f1tl.append(f1t);acctl.append(acct);pretl.append(pret);rectl.append(rect)
    f1tml.append(f1tm);pretml.append(pretm);rectml.append(rectm);auctl.append(auct)
    print("patient:",p)
    print("test accuracy:",acct)
    print("test f1 macro:",f1t)
    print("test f1 micro:",f1tm)
    print("test precision macro:",pret)
    print("test precision micro:",pretm)
    print("test recall macro:",rect)
    print("test recall micro:",rectm)
    print("test auc:",auct)
    print()


ZeroDivisionError: ignored

In [ ]:
print("average test accuracy:",np.mean(acctl))
print("average test f1 macro:",np.mean(f1tl))
print("average test f1 micro:",np.mean(f1tml))
print("average test precision macro:",np.mean(pretl))
print("average test precision micro:",np.mean(pretml))
print("average test recall macro:",np.mean(rectl))
print("average test recall micro:",np.mean(rectml))
print("average test auc:",np.mean(auctl))

average test accuracy: 0.7356884057971015
average test f1 macro: 0.7058862433862435
average test f1 micro: 0.7356884057971015
average test precision macro: 0.7299074074074073
average test precision micro: 0.7356884057971015
average test recall macro: 0.7347222222222224
average test recall micro: 0.7356884057971015
average test auc: 0.9473842075448881
